# Практическое задание 2 (часть 1)

# Определение частей речи с помощью скрытой марковской модели

## курс "Математические методы анализа текстов"


### ФИО: <впишите>

## Введение

### Постановка задачи

В данной лабораторной работе вам предстоит обучить скрытую марковскую модель на размеченных данных и реализовать алгоритм Витерби для задачи POS-теггинга (определение частей речи слов в тексте), а также ознакомиться с использованием  ряда POS-теггеров из библиотеки NLTK.

### Комментарии и советы

1. Для выполнения потребуются модули Python numpy, nltk.

2. Все необходимые для выполнения задания данные либо приложены, либо могут быть скачаны с помощью nltk.download().

3. Посмотреть параметры конструктора и других методов классов можно набрав и выполнив в ячейке с кодом '?full_method_name'.

### Задача определения частей речи (POS)

Мы будем решать задачу определения частей речи (POS-теггинга) с помощью скрытой марковской модели (HMM). Формула совместной плотности наблюдаемых и скрытых переменных задается как

$$ p(x, t) = p(t) p(x|t) = p(t_1)  \prod_{i=2}^{N_x} p(t_i|t_{i-1}) \prod_{i=1}^{N_x} p(x_i|t_i)$$

#### Переменные модели

- наблюдаемые переменные $X$ - словарь корпуса;

- скрытые переменные $T$ - множество POS-тегов.

- x - одно предложение, $N_x$ - длина предложения

- t - теги одного предложения, $N_t$ - длина вектора меток

#### Параметры модели

- матрица вероятностей переходов $A \in \mathbb{R}^{|T| \times |T|}$, $A_{ij} = p(t_s=i|t_{s-1}=j) \; \forall s$

- матрица выходных вероятностей $B \in \mathbb{R}^{|X| \times |T|}$, $B_{ij} = p(x_s =i|t_s =j) \; \forall s$

- вектор начальных вероятностей $C \in \mathbb{R}^{|T|}$, $C_i = p(t_1=i)$


#### Обучение модели

* Для обучения параметров $A$ и $B$ используется метод максимума правдоподобия. Оценки вычисляются на основе частот совстречаемости тегов и тегов со словами:

$$a_{ij} = \frac{\sum_{t}\sum_{s=2}^{N_t} \mathbb{I}[t_{s} = i, t_{s - 1} = j]}{\sum_{t}\sum_{s=2}^{N_t} \mathbb{I}[t_{s-1} = j]}$$

$$b_{ij} = \frac{\sum_{t, x}\sum_{s=1}^{N_t} \mathbb{I}[x_{s} = i, t_{s} = j]}{\sum_{t, x}\sum_{s=1}^{N_t} \mathbb{I}[t_{s} = j]}$$

* Параметры $C$ можно аналогично вычислять по частотам или считать распределение $p(t_1)$ равномерным

#### Применение модели

Применение модели на новых данных реализуется с помощью алгоритма Витерби.Это алгоритм динамиеского программирования, с помощью которого мы будем находить наиболее вероятную последовательность скрытых состояний модели для фиксированной последовательности слов:

$$ \hat{t} = \arg \max_{t} p(t|x) = \arg \max_{t} p(x, t) $$

Определим функцию, определяющую максимальную вероятность последовательности, заканчивающейся на $i$-ой позиции в состоянии $k$:

$$\delta(k, i) = \max_{t_1, \dots t_{i-1}} p(x_1, \dots x_i, t_1, \dots t_i=k)$$

Тогда $\max_{k} \delta(k, N_x)$ - максимальная вероятность всей последовательности. А состояния, на которых эта вероятность достигается - ответ задачи.

Алгоритм Витерби заключается в последовательном пересчете функции $\delta(k, i)$ по формуле:

$$\delta(k, i) = \max_{m} \delta(m, i-1) p(t_i = k|t_{i-1} = m) p(x_i|t_i=k) $$

Аналогично пересчитывается функция, определяющая, на каком состоянии этот максимум достигается:

$$s(k, i) = \arg \max_{m} \delta(m, i-1) p(t_i = k|t_{i-1} = m) p(x_i|t_i=k) $$


На практике это означает заполнение двумерных массивов размерности: (длина последовательности) $\times$ (количество возможных состояний). Когда массивы заполнены, $\arg \max_{k} \delta(k, N_x)$ говорит о последнем состоянии. Начиная с него можно восстановить все состояния по массиву $s$. 

Осталось уточнить, как стартовать последовательный пересчет (чем заполнить первый столбец массива вероятностей):

$$\delta(k, 1) = p(k) p(x_1|t_1=k)$$

Подробнее о HMM можно прочитать по [ссылке](https://web.stanford.edu/~jurafsky/slp3/A.pdf)

## Часть 1. Загрузка корпуса (1 балл)

Загрузите brown корпус с универсальной системой тегирования. Для этого вам понадобятся ресурсы brown и universal_tagset из nltk.download().  

Так как каждый конкретный датасет может использовать свою систему тегов, в NLTK предусмотрено универсальное множество тегов и возможность приведения к нему других систем. Это множество включает в себя следующие теги:


- ADJ - adjective (new, good, high, ...)
- ADP - adposition	(on, of, at, ...)
- ADV - adverb	(really, already, still, ...)
- CONJ	- conjunction	(and, or, but, ...)
- DET - determiner, article	(the, a, some, ...)
- NOUN	- noun	(year, home, costs, ...)
- NUM - numeral	(twenty-four, fourth, 1991, ...)
- PRT -	particle (at, on, out, ...)
- PRON - pronoun (he, their, her, ...)
- VERB - verb (is, say, told, ...)
- .	- punctuation marks	(. , ;)
- X	- other	(ersatz, esprit, dunno, ...)

Обратите внимание, что тегсеты в корпусах текстов и в различных теггерах могут быть разными. Проверять это можно, глядя на сами теги, а симптом - подозрительно низкое качество теггирования. В таких случаях рекомендуется всё приводить сперва к универсальному тегсету, а потом уже мерять качество. Полезной может оказаться эта ссылка http://www.nltk.org/_modules/nltk/tag/mapping.html

In [ ]:
from nltk.corpus import brown

brown_tagged_sents = brown.tagged_sents(tagset="universal")

# you code here

Проанализируйте данные, с которыми Вы работаете. В частности, ответьте на вопросы:
- Каков общий объем датасета, формат?
- Приведены ли слова к нижнему регистру? Чем  это нам может в дальнейшем помешать?
- Как распределены слова в корпусе?  Как распределены теги в корпусе? Подсчитайте частоты и отобразите любым удобным для Вас способом. Проинтерпретируйте полученные результаты.

In [ ]:
# your answers here

Cделайте случайное разбиение выборки на обучение и контроль в отношении 9:1.Если впоследствии обучение моделей будет занимать слишком много времени, работайте с подвыборкой, например, только текстами определенных категорий.

In [ ]:
# your code here

## Часть 2. Скрытая марковская модель (4 балла) 

### Метод максимального правдоподобия для обучения модели

Требуется построить скрытую марковскую модель (class HiddenMarkovModel) и написать метод fit для настройки всех её параметров с помощью оценок максимального правдоподобия по размеченным данным (последовательности пар слово+тег):

- Вероятности переходов между скрытыми состояниями $p(t_i | t_{i - 1})$ посчитайте на основе частот биграмм POS-тегов.

- Вероятности эмиссий наблюдаемых состояний $p(x_i | t_i)$ посчитайте на основе частот "POS-тег - слово".

- Обратите внимание на проблему разреженности счетчиков и сделаейте все вероятности сглаженными по Лапласу ([add-k smoothing](https://en.wikipedia.org/wiki/Additive_smoothing)).

- Распределение вероятностей начальных состояний $p(t_1)$ задайте равномерным.


### Алгоритм Витерби для применения модели

Требуется написать метод .predict для определения частей речи на тестовой выборке. Чтобы использовать обученную модель на новых данных, необходимо реализовать алгоритм Витерби. 

В реализации рекомендуется перейти к логарифмам, т.к. произведение большого числа маленьких вероятностей может приводить к вычислительным ошибкам.

In [ ]:
class HiddenMarkovModel:    
    def __init__(self, k_smoothing=1.0):
        """
        k_smoothing : float, constant in add-k-smoothing
        """
        self.k_smoothing = k_smoothing
        
    def fit(self, train_tokens_tags_list):
        """
        Fit the model using maximum likelihood method.
        
        train_tokens_tags_list: list of list of pairs (token, tag) 
        """
        pass
    
    def predict(self, test_tokens_list):
        """
        Return predictions for test_tokens_list using viterbi algorithm.
        
        test_tokens_list : list of list of tokens
        
        return: list of list of tags
        """
        pass

Обучите скрытую марковскую модель:

In [ ]:
# your code here

Проверьте работу реализованного алгоритма на следующих модельных примерах, проинтерпретируйте результат.

- 'he can stay'
- 'a milk can'
- 'i saw a dog'
- 'an old saw'

In [ ]:
# your code here

Примените модель к отложенной выборке Брауновского корпуса и подсчитайте точность (accuracy) определения тегов (ожидаемая точность $\approx 0.93$). Сделайте выводы. 

In [ ]:
# your code here

## Бонусная часть. Сравнение с готовыми POS-теггерами из NLTK (2 балла)

В прошлом пункте Вы реализовали свой POS-тегер на основе скрытой марковской модели. Теперь сравните его работу с готовыми средставми, доступными в библиотеке NLTK: http://www.nltk.org/api/nltk.tag.html

Сравните с вашей моделью любые из 4-х теггеров, представленных ниже.

При проведении экспериментов обращайте внимание на следующие моменты (и отразите их в отчете):
- Какой подход лежит в основе теггера
- На каких данных он обучен (если Вы скачали готовую модель)
- Сколько времени занимает обучение на brown корпусе (если обучаете сами)
- Какая точность получается на контролькой выборке

Сформируйте рекоммендиции о том, какую технологию Вы бы использовали, если встретитесь с задачей определения частей речи в будущем.

In [ ]:
import nltk
from nltk.tag.mapping import map_tag

### 1. DefaultTagger
Простая заглушка, ставящая всем словам один и тот же pos-тег. Очевидно, для максимизации качества, мы хотим выбрать самую частотную метку из всех меток обучающей выборки, т.е. метку 'NOUN'.

In [ ]:
from nltk.tag import DefaultTagger
default_tagger = DefaultTagger(u'NOUN')

# your code here

### 2. RegexpTagger

Теггер, который присваивает слову часть речи, основываясь на регулярных выражениях. Например, ставить слову метку 'NOUN', если слово кончается на 'ness'. Ниже приведен простой пример возможных правил. В качестве backoff теггера использован DefaultTagger.



In [ ]:
from nltk.tag import RegexpTagger

regexp_tagger = RegexpTagger(regexps=[(r'^-?[0-9]+(.[0-9]+)?$', 'CD'),   # cardinal numbers
                                      (r'(The|the|A|a|An|an)$', 'AT'),   # articles
                                      (r'.*able$', 'JJ'),                # adjectives
                                      (r'.*ness$', 'NN'),                # nouns formed from adjectives
                                      (r'.*ly$', 'RB'),                  # adverbs
                                      (r'.*s$', 'NNS'),                  # plural nouns
                                      (r'.*ing$', 'VBG'),                # gerunds
                                      (r'.*ed$', 'VBD'),                 # past tense verbs
                                      (r'.*', 'NN')                      # nouns (default)
                                     ],
                             backoff=default_tagger)

# your code here
# use map_tag() to tranform 'en-ptb' to 'universal' tags.

### 3. N-грамные теггеры

В теггерах, основанных на n-граммах,  принятие решения происходит в зависимости от $n-1$ предыдущих слов и их тегов. Эти теггеры необходимо обучать по размеченной обучающей коллекции. 

Заметим, что TrigramTagger и BigramTagger работают очень плохо без указания backoff. Поэтому предлагается построить композицию, где 
в качестве backoff для UnigramTagger использовать DefaultTagger, для BigramTagger использовать UnigramTagger, для TrigramTagger  использовать BigramTagger. 

In [ ]:
from nltk.tag import UnigramTagger
from nltk.tag import BigramTagger
from nltk.tag import TrigramTagger

# your code here

### 4. Stanford tagger

Скачайте предобученную модель от Стэнфорда: https://nlp.stanford.edu/software/tagger.shtml и примените к тестовым данным. 
Не забудьте преобразовать систему тэгов из 'en-ptb' в 'universal' с помощью функции map_tag.

In [ ]:
from nltk.tag.stanford import StanfordPOSTagger

# Add the jar and model via their path:
jar = 'you_path_here/stanford-postagger-3.9.1.jar'
model = 'your_path_here/models/english-bidirectional-distsim.tagger'
stanford_tagger = StanfordPOSTagger(model, jar, encoding='utf8')

# your code here

### 5. Теггеры из NLTK на основе графических моделей

Обучите теггер, основанный на HMM или CRF, на основе класса из nltk.

In [ ]:
from nltk.tag import HiddenMarkovModelTagger, CRFTagger

# your code here

### Сравнение моделей

Сравните различные модели по качеству, сделайте выводы.

In [ ]:
# your code here